In [ ]:
import pandas as pd
import numpy as np
import html
import re, string, unicodedata
import nltk
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

# Drop, Clean & Split


In [ ]:
df = pd.read_csv("data/hn_all_w_sentiment_cleaned_inplace.csv")
df.head()
df.isnull().sum()

In [3]:
df = df[["commentid","cleaned_comment"]]
df.shape

(15397309, 2)

In [4]:
df.isnull().sum()
df = df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

In [5]:
df.isnull().sum()

commentid          0
cleaned_comment    0
dtype: int64

In [6]:
df = df.fillna("NULL VALUE PLACEHOLDER COMMENT")

In [7]:
df.isnull().sum()

commentid          0
cleaned_comment    0
dtype: int64

In [8]:
!mkdir data4

mkdir: cannot create directory ‘data4’: File exists


In [ ]:
a //10

In [12]:
a, b = df.shape
inc = a //10

c = inc*1
d = inc*2
e = inc*3
f = inc*4
g = inc*5
h = inc*6
i = inc*7
j = inc*8
k = inc*9
l = inc*10

df_a = df[0:c]
df_b = df[c:d]
df_c = df[d:e]
df_d = df[e:f]
df_e = df[f:g]
df_f = df[g:h]
df_g = df[h:i]
df_h = df[i:j]
df_i = df[j:k]
df_j = df[k:l]
df_k = df[l:]

In [14]:
df_d.to_csv('data4/hn_cleaned_ready_for_bert_a_11.csv',index=False)

In [ ]:
for value in [c,d,e,f,g,h,i,j,k,l]:
    print(value)

In [ ]:
df_a.to_csv('data4/hn_cleaned_ready_for_bert_a1.csv',index=False)
df_b.to_csv('data4/hn_cleaned_ready_for_bert_a2.csv',index=False)
df_c.to_csv('data4/hn_cleaned_ready_for_bert_a3.csv',index=False)
df_d.to_csv('data4/hn_cleaned_ready_for_bert_a11.csv',index=False)
df_e.to_csv('data4/hn_cleaned_ready_for_bert_a4.csv',index=False)
df_f.to_csv('data4/hn_cleaned_ready_for_bert_a5.csv',index=False)
df_g.to_csv('data4/hn_cleaned_ready_for_bert_a6.csv',index=False)
df_h.to_csv('data4/hn_cleaned_ready_for_bert_a7.csv',index=False)
df_i.to_csv('data4/hn_cleaned_ready_for_bert_a8.csv',index=False)
df_j.to_csv('data4/hn_cleaned_ready_for_bert_a9.csv',index=False)
df_k.to_csv('data4/hn_cleaned_ready_for_bert_a10.csv',index=False)
df_l.to_csv('data4/hn_cleaned_ready_for_bert_a11.csv',index=False)

In [ ]:
CHARS_TO_REMOVE = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—1234567890'

In [ ]:
# df_processing = 1
# for each_df in list_of_dfs:
#     each_df['cleaned_comment'] = each_df["cleaned_comment"].apply(lambda x: text.text_to_word_sequence(x, filters=CHARS_TO_REMOVE, lower=True, split=' '))
#     print("Processed Sub_df # %s" % df_processing)
#     df_processing += 1
df['cleaned_comments'] = df["cleaned_comment"].apply(lambda x: text.text_to_word_sequence(x, filters=CHARS_TO_REMOVE, lower=True, split=' '))

In [ ]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    return df

df = standardize_text(df, 'cleaned_comment')
df.isnull().sum()


In [ ]:

df['cleaned_comment'] = df["cleaned_comment"].apply(lambda x: text.text_to_word_sequence(x, filters=CHARS_TO_REMOVE, lower=True, split=' '))

In [ ]:
df.head()

In [ ]:
#cleaned_words = [word for tokens in df['SentimentText'] for word in tokens]
cleanedVOCAB = sorted(list(set(cleaned_words)))
display("%s words total, with a vocabulary size of %s" % (len(cleaned_words), len(cleanedVOCAB)))

display(df.head(20))

# Part 3 - Document Classification

1) Use Train_Test_Split to create train and test datasets.

2) Vectorize the tokenized documents using your choice of vectorization method. 

 - Stretch goal: Use both of the methods that we talked about in class.

3) Create a vocabulary using the X_train dataset and transform both your X_train and X_test data using that vocabulary.

4) Use your choice of binary classification algorithm to train and evaluate your model's accuracy. Report both train and test accuracies.

 - Stretch goal: Use an error metric other than accuracy and implement/evaluate multiple classifiers.



## Bag of Words w/ Count Vectorizer

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
seed = 42

In [ ]:
# Defining the Count Vectorizer function. 
def cv(data):
    count_vectorizer = CountVectorizer()
    emb = count_vectorizer.fit_transform(data)
    return emb, count_vectorizer

# Here I define my corpus and lables. These are used throughout all the other models/processes.
list_corpus = df["SentimentText"].apply(lambda y: " ".join(str(x) for x in y)).tolist()
list_labels = df["Sentiment"].tolist()

X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.2, 
                                                                                random_state=seed)

X_train_counts, count_vectorizer = cv(X_train)
X_test_counts = count_vectorizer.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=40)
clf.fit(X_train_counts, y_train)

y_predicted_counts = clf.predict(X_test_counts)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

def get_metrics(y_test, y_predicted):  
    # true positives / (true positives+false positives)
    precision = precision_score(y_test, y_predicted, pos_label=None,
                                    average='weighted')             
    # true positives / (true positives + false negatives)
    recall = recall_score(y_test, y_predicted, pos_label=None,
                              average='weighted')
    
    # harmonic mean of precision and recall
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')
    
    # true positives + true negatives/ total
    accuracy = accuracy_score(y_test, y_predicted)
    return accuracy, precision, recall, f1

accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
MNB = MultinomialNB().fit(X_train_counts, y_train)

train_predictions = MNB.predict(X_train_counts)
test_predictions = MNB.predict(X_test_counts)

accuracy, precision, recall, f1 = get_metrics(y_test, test_predictions)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

## Now BoW TF-IDF Vectorizer

In [ ]:
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer()

    train = tfidf_vectorizer.fit_transform(data)

    return train, tfidf_vectorizer

X_train_tfidf, tfidf_vectorizer = tfidf(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
clf_tfidf = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', n_jobs=-1, random_state=40)
clf_tfidf.fit(X_train_tfidf, y_train)

y_predicted_tfidf = clf_tfidf.predict(X_test_tfidf)

In [ ]:
accuracy_tfidf, precision_tfidf, recall_tfidf, f1_tfidf = get_metrics(y_test, y_predicted_tfidf)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy_tfidf, precision_tfidf, 
                                                                       recall_tfidf, f1_tfidf))

# Part 4 -  Word2Vec

1) Fit a Word2Vec model on your cleaned/tokenized twitter dataset. 

2) Display the 10 words that are most similar to the word "twitter"

## Word2Vec

In [ ]:
model = Word2Vec(df["SentimentText"].tolist(), min_count=1, size=5)

In [ ]:
print(model)
print(list(model.wv.vocab))
print(len(model.wv.vocab))

In [ ]:
model.wv.most_similar('twitter')